## Data
### 2016 Eora26

## Export and Import
- TID: Total Intermediate demand;
- TFD: Total Final demand;
- HID: Home Intermediate demand;
- FID: Foreign Intermediate demand;
- HFD: Home Final demand;
- FFD: Foreign Final demand.

#### Export calculation method:
1. TID: Intermediate matrix sum by rows; TFD: Final demand matrix sum by rows.
2. Trade mask (a matrix with the same size as Intermediate matrix or Final demand matrix, which has zeros for domestic flow and 1 for other flows); Multiply trade mask by Intermediate matrix or Final demand matrix elementwisely to derive masked Intermediate matrix or Final demand matrix; A masked matrix only contains export flows.
3. FID: Masked Intermediate matrix sum by rows; FFD: Masked Final demand matrix sum by rows.
4. HID = TID - FID; HFD = TFD - FFD

#### Import calculation method:
- Import into industries: Masked Intermediate matrix sum by columns;
- Import by final demand: Masked Final demand matrix sum by columns.

#### Check correctness:
- **Gross input must be equal gross output (xin = xout).**

In [1]:
import pandas as pd
import numpy as np
import json
import copy

In [2]:
# Read files
## Final demand matrix
f = open('C:/Users/15311/Desktop/Global_PM2.5/EURO/EURO26/Eora26_2016_bp_FD.txt','r')
content = f.readlines()
newData = [0 for i in range(len(content))]
for (i, line) in enumerate(content):
    newData[i] = line.split("\t")
    newData[i][-1] = newData[i][-1][:-1]
FD_df = pd.DataFrame(newData)
f.close()

f = open('C:/Users/15311/Desktop/Global_PM2.5/EURO/EURO26/labels_T.txt','r')
content = f.readlines()
newData = [0 for i in range(len(content))]
for (i, line) in enumerate(content):
    newData[i] = line.split("\t")[:-1]
LABELS_df = pd.DataFrame(newData)
f.close()

## Intermediate transaction matrix
f = open('C:/Users/15311/Desktop/Global_PM2.5/EURO/EURO26/Eora26_2016_bp_T.txt','r')
content = f.readlines()
newData = [0 for i in range(len(content))]
for (i, line) in enumerate(content):
    newData[i] = line.split("\t")
    newData[i][-1] = newData[i][-1][:-1]
T_df = pd.DataFrame(newData)
f.close()

In [3]:
# From str to float : T_df, FD_df
T_df = T_df.astype('float')
FD_df = FD_df.astype('float');

In [4]:
T_df # transaction matrix

,0,1,2,3,4,5,6,7,8,9,...,4905,4906,4907,4908,4909,4910,4911,4912,4913,4914
0,98135.00000,25.30470,359.37200,214505.00000,2674.45000,32478.10000,2813.46000,139.37000,186.41400,19.09510,...,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.430900e-07,1.000000e-07,1.000000e-07,398916.00
1,132.40800,418.24000,2.32455,28665.90000,4.19821,1.56355,35.82680,18.74070,2.47318,1.07425,...,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,28619.20
2,1603.91000,37.79110,10299.00000,667.63800,68.57620,698.94000,62054.80000,6174.97000,188.68600,265.54800,...,1.048770e-07,1.043120e-07,1.042980e-07,1.000000e-07,1.001720e-07,1.002060e-07,1.000000e-07,1.000000e-07,1.000000e-07,37647.30
3,40408.80000,468.59600,54.37240,161090.00000,1532.57000,1237.13000,4558.57000,28.90200,54.17510,13.63440,...,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,5095.01
4,1706.29000,219.67600,385.27000,1929.72000,50932.00000,4957.07000,6176.40000,919.84400,4939.74000,3683.98000,...,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,1.000000e-07,20594.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4910,3.01483,3.20941,4.24502,5.19153,5.24824,5.24130,5.16952,5.18477,5.10443,5.17863,...,1.097940e+00,7.491060e-01,7.833540e-01,3.013900e-01,4.209160e-01,1.014600e-01,3.243420e+01,2.166330e+01,2.714370e+01,367530.00
4911,1.17719,1.37551,1.84375,2.25750,2.27674,2.29281,2.26491,2.26818,2.24488,2.25501,...,9.593950e-02,1.035910e-01,9.461920e-02,1.078180e-01,9.560480e-02,9.890780e-02,1.014600e-01,5.840000e-01,4.603710e-01,40930.00
4912,3.30335,3.58883,4.79122,5.92975,5.93691,5.97308,5.90872,5.91886,5.85291,5.88518,...,7.730420e-02,8.454030e-02,7.691460e-02,8.448920e-02,7.893080e-02,7.932750e-02,7.433410e-01,1.014600e-01,5.172770e-01,145728.00
4913,5.94325,6.83916,8.99569,11.27200,11.27610,11.35070,11.24740,11.25540,11.12770,11.17790,...,8.463560e-02,9.186710e-02,8.350150e-02,9.163710e-02,8.236980e-02,8.578660e-02,1.438470e+00,5.986140e-01,1.014600e-01,179240.00


In [5]:
FD_df #final demand matrix

,0,1,2,3,4,5,6,7,8,9,...,1130,1131,1132,1133,1134,1135,1136,1137,1138,1139
0,2.441240e+06,21191.40000,4170.41000,17477.90000,9921.32000,50.29780,1.369080,1.338850,2.235760,2.504540,...,1.067240,1.420560,0.030502,1.050150,0.0,0.0,0.0,0.0,0.0,0.0
1,9.756520e+04,526.74900,2603.48000,7.44061,793.11800,7.44061,0.210492,0.210492,0.244429,0.255776,...,0.709638,0.598005,0.020051,0.598005,0.0,0.0,0.0,0.0,0.0,0.0
2,1.923700e+04,18.99120,334.61100,548.32100,1452.70000,1.78847,0.156236,0.156236,0.246782,0.215100,...,0.571534,0.398656,0.020445,0.398656,0.0,0.0,0.0,0.0,0.0,0.0
3,8.358320e+05,11606.10000,1.04009,0.44913,1812.68000,0.44913,0.056894,0.056894,0.080191,0.070835,...,0.398171,0.280365,0.015234,0.280365,0.0,0.0,0.0,0.0,0.0,0.0
4,1.393990e+05,1849.85000,28.03080,5638.92000,329.13900,13.14740,0.097453,0.097453,0.173592,0.149557,...,0.425890,0.348862,0.022607,0.348862,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4910,6.406050e+00,6.40605,7.61509,7.99647,3.62230,7.99647,5.176600,5.176600,5.871570,6.588000,...,301040.000000,71759.100000,0.000000,71759.100000,0.0,0.0,0.0,0.0,0.0,0.0
4911,2.969850e+00,2.96985,3.40420,3.24971,1.71085,3.24971,1.817520,1.817520,1.933540,2.008380,...,109792.000000,21763.100000,0.000000,21763.100000,0.0,0.0,0.0,0.0,0.0,0.0
4912,7.513970e+00,7.51397,8.74698,8.27024,3.83962,8.27024,5.790060,5.790060,6.324450,6.546780,...,117676.000000,22373.000000,0.000000,22373.000000,0.0,0.0,0.0,0.0,0.0,0.0
4913,1.449490e+01,14.49490,17.07700,15.95110,6.92056,15.95110,11.069300,11.069300,12.226400,12.927700,...,134683.000000,25011.100000,0.000000,25011.100000,0.0,0.0,0.0,0.0,0.0,0.0


In [216]:
# drop ANT
T_df = T_df.drop(columns=list(range(120*26,121*26)) + list(range(174*26,175*26)))
T_df = T_df.drop(list(range(120*26,121*26)) + list(range(174*26,175*26)))
FD_df = FD_df.drop(columns=list(range(120*6,121*6)) + list(range(174*6,175*6))) 
FD_df = FD_df.drop(list(range(120*26,121*26)) + list(range(174*26,175*26))) 
LABELS_df = LABELS_df.drop(list(range(120*26,121*26)) + list(range(174*26,175*26)))

# drop USR
# T_df = T_df.drop(columns=list(range(174*26,175*26))) 
# T_df = T_df.drop(list(range(174*26,175*26)))
# FD_df = FD_df.drop(columns=list(range(174*6,175*6)))
# FD_df = FD_df.drop(list(range(174*26,175*26))) 
# LABELS_df = LABELS_df.drop(list(range(174*26,175*26)))

## Remember not to reset index while dropping columns!!!

In [219]:
175*26 in T_df.columns

True

In [220]:
T_df = T_df.reset_index(drop=True)
FD_df = FD_df.reset_index(drop=True)
LABELS_df = LABELS_df.reset_index(drop=True)

In [221]:
T_df.columns = list(range(0,4863))
FD_df.columns = list(range(0,1128))

In [222]:
# Extact names of countries
LABELS_df.columns = ["Country","c","I","Industries"]
Export = pd.DataFrame(LABELS_df,columns=["Country","Industries"])
Countries = list(LABELS_df["Country"].unique())

In [223]:
'USR' in Countries

False

### Export

In [224]:
# Calculate TID and TFD
Export["TID"] = T_df.sum(axis=1) # Total intermediate demand
Export["TFD"] = FD_df.sum(axis=1) # Total final demand

In [225]:
# Calculate trade mask matrix
T_df1 = copy.copy(T_df)
FD_df1 = copy.copy(FD_df)
Masked_T = T_df1.to_numpy()
Masked_FD = FD_df1.to_numpy()

In [226]:
Masked_FD.shape,Masked_T.shape

((4863, 1128), (4863, 4863))

In [227]:
# A masked matrix only contains export flows: zeros for domestic flow.
for i in range(187):
    Masked_T[i*26:(i+1)*26,i*26:(i+1)*26] = 0
    Masked_FD[i*26:(i+1)*26,i*6:(i+1)*6] = 0
Masked_T[-1,-1] = 0
Masked_FD[-1,-6:] = 0

Masked_T_df = pd.DataFrame(Masked_T)
Masked_FD_df = pd.DataFrame(Masked_FD)

In [228]:
# Calculate FID and FFD
Export["FID"] = Masked_T_df.sum(axis=1) # Foreign intermediate demand
Export["FFD"] = Masked_FD_df.sum(axis=1) # Foreign final demand

In [229]:
Export

,Country,Industries,TID,TFD,FID,FFD
0,AFG,Agriculture,1.538228e+06,2.982173e+06,1.118930e+06,488121.500819
1,AFG,Fishing,8.204556e+04,1.021952e+05,3.288965e+04,691.735172
2,AFG,Mining and Quarrying,2.571376e+05,2.229511e+04,4.083923e+04,701.700171
3,AFG,Food & Beverages,4.822587e+05,8.496689e+05,6.115531e+03,416.200579
4,AFG,Textiles and Wearing Apparel,1.526739e+05,1.494905e+05,2.357414e+04,2232.457909
...,...,...,...,...,...,...
4858,ZWE,"Education, Health and Other Services",4.053929e+05,9.166833e+05,4.051682e+05,10999.052329
4859,ZWE,Private Households,5.454534e+04,3.839635e+05,5.453544e+04,2445.266074
4860,ZWE,Others,1.878119e+05,3.851117e+05,1.878020e+05,7345.654559
4861,ZWE,Re-export & Re-import,2.625942e+05,4.400673e+05,2.625817e+05,14370.062704


In [230]:
# Calculate HID and HFD
Export["HID"] = Export["TID"] - Export["FID"] # Home intermediate demand
Export["HFD"] = Export["TFD"] - Export["FFD"] # Home final demand

In [231]:
# Gross output and Export
Export["Gross_output"] = Export["TID"] + Export["TFD"]
Export["Tol_export"] = Export["FID"] + Export["FFD"]

In [232]:
# Calculate total export of each country in a dict.
Tol_export = dict(Export.groupby(by=["Country"])["Tol_export"].sum())

In [233]:
# Simple check
Export[Export["Country"] == "ROW"]["Tol_export"].sum() == Tol_export["ROW"]

True

In [234]:
Tol_export["PSE"]

2566482.0274316235

In [235]:
Export[Export['TFD'] <0]

,Country,Industries,TID,TFD,FID,FFD,HID,HFD,Gross_output,Tol_export
28,ALB,Mining and Quarrying,4.019093e+05,-5.244373e+03,7.067531e+04,4.988982e+03,3.312340e+05,-1.023335e+04,3.966650e+05,7.566429e+04
37,ALB,Recycling,3.351214e+04,-3.453328e+03,2.271360e+04,5.109610e+03,1.079854e+04,-8.562937e+03,3.005881e+04,2.782321e+04
444,BEL,Mining and Quarrying,7.120145e+06,-4.346294e+04,6.444965e+06,4.453133e+04,6.751805e+05,-8.799427e+04,7.076682e+06,6.489496e+06
548,BTN,Mining and Quarrying,7.347791e+04,-3.166672e+02,2.373465e+04,6.655362e+02,4.974326e+04,-9.822035e+02,7.316124e+04,2.440018e+04
553,BTN,Metal Products,1.063034e+05,-3.063535e+03,9.901462e+04,2.251341e+03,7.288777e+03,-5.314875e+03,1.032399e+05,1.012660e+05
...,...,...,...,...,...,...,...,...,...,...
4323,THA,Metal Products,3.658781e+07,-7.853084e+06,1.219838e+07,1.173894e+06,2.438943e+07,-9.026978e+06,2.873473e+07,1.337228e+07
4448,TUR,Mining and Quarrying,9.818269e+06,-9.806427e+05,5.021859e+06,4.942458e+04,4.796410e+06,-1.030067e+06,8.837626e+06,5.071284e+06
4628,USA,Agriculture,3.931916e+08,-1.794300e+07,4.143433e+07,1.826044e+07,3.517573e+08,-3.620345e+07,3.752486e+08,5.969478e+07
4630,USA,Mining and Quarrying,2.889989e+08,-7.527389e+07,2.634307e+07,5.771048e+05,2.626559e+08,-7.585100e+07,2.137250e+08,2.692017e+07


In [236]:
Export[Export['Country'] == 'CHN']

,Country,Industries,TID,TFD,FID,FFD,HID,HFD,Gross_output,Tol_export
1014,CHN,Agriculture,8.085452e+08,6.108391e+08,1.746100e+07,4.733235e+06,7.910842e+08,6.061059e+08,1.419384e+09,2.219424e+07
1015,CHN,Fishing,7.753417e+07,7.787804e+07,1.281760e+06,5.329218e+05,7.625241e+07,7.734512e+07,1.554122e+08,1.814682e+06
1016,CHN,Mining and Quarrying,8.063941e+08,3.465298e+07,1.883922e+07,3.365811e+05,7.875549e+08,3.431640e+07,8.410471e+08,1.917580e+07
1017,CHN,Food & Beverages,6.483983e+08,7.657731e+08,2.550391e+07,2.358290e+07,6.228944e+08,7.421902e+08,1.414171e+09,4.908681e+07
1018,CHN,Textiles and Wearing Apparel,8.092073e+08,4.478151e+08,1.322958e+08,1.953401e+08,6.769116e+08,2.524750e+08,1.257022e+09,3.276359e+08
1019,CHN,Wood and Paper,5.515575e+08,6.553861e+07,3.556892e+07,1.456933e+07,5.159886e+08,5.096928e+07,6.170962e+08,5.013825e+07
1020,CHN,"Petroleum, Chemical and Non-Metallic Mineral P...",3.046697e+09,2.706941e+08,2.382738e+08,4.065894e+07,2.808424e+09,2.300351e+08,3.317391e+09,2.789328e+08
1021,CHN,Metal Products,2.289517e+09,1.061599e+08,1.594622e+08,1.885857e+07,2.130055e+09,8.730135e+07,2.395677e+09,1.783208e+08
1022,CHN,Electrical and Machinery,2.315540e+09,1.553307e+09,4.151321e+08,2.972566e+08,1.900408e+09,1.256050e+09,3.868847e+09,7.123887e+08
1023,CHN,Transport Equipment,7.005290e+08,4.610633e+08,2.769236e+07,3.066631e+07,6.728367e+08,4.303970e+08,1.161592e+09,5.835867e+07


In [237]:
Export.to_csv("Export_woANTUSR.csv",index=False) 

### Import

In [238]:
Import_ind = Masked_T_df.sum(axis=0) # Import into industries
Import_fd = Masked_FD_df.sum(axis=0) # Import by final demand

In [239]:
Import_ind.shape,Import_fd.shape

((4863,), (1128,))

In [240]:
Tol_ind = {}
for i in range(187):
    Tol_ind[Countries[i]] = sum(Import_ind[i*26:(i+1)*26])
Tol_ind["ROW"] = Import_ind[len(Import_ind)-1]

In [241]:
Tol_fd = {}
for i in range(188):
    Tol_fd[Countries[i]] = sum(Import_fd[i*6:(i+1)*6])

In [242]:
Tol_import = {}
for i in Countries:
    Tol_import[i] = Tol_fd[i] + Tol_ind[i]

## Leontief Inverse

In [243]:
T_df2 = copy.copy(T_df)
Transaction_matrix = T_df2.to_numpy()
gross_output = list(Export["Gross_output"])

In [244]:
Transaction_matrix

array([[9.81350e+04, 2.53047e+01, 3.59372e+02, ..., 1.00000e-07,
        1.00000e-07, 3.98916e+05],
       [1.32408e+02, 4.18240e+02, 2.32455e+00, ..., 1.00000e-07,
        1.00000e-07, 2.86192e+04],
       [1.60391e+03, 3.77911e+01, 1.02990e+04, ..., 1.00000e-07,
        1.00000e-07, 3.76473e+04],
       ...,
       [3.30335e+00, 3.58883e+00, 4.79122e+00, ..., 1.01460e-01,
        5.17277e-01, 1.45728e+05],
       [5.94325e+00, 6.83916e+00, 8.99569e+00, ..., 5.98614e-01,
        1.01460e-01, 1.79240e+05],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00]])

In [245]:
# Small example
a = np.array([[1000,2000,3000],[4000,5000,6000],[7000,8000,9000]])
b = [10.0,100.0,1000.0]
for i in range(3):
    a[:, i] = a[:, i] / b[i]
print (a)

[[100  20   3]
 [400  50   6]
 [700  80   9]]


In [246]:
np.identity(3) - a

array([[ -99.,  -20.,   -3.],
       [-400.,  -49.,   -6.],
       [-700.,  -80.,   -8.]])

In [247]:
# A
A = np.zeros(shape=(4863,4863))
for i in range(4863):
    if gross_output[i] == 0:
        A[:, i] = 0
        continue
    A[:, i] = Transaction_matrix[:, i] / gross_output[i] 

In [248]:
np.identity(4863)

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [249]:
# (I-A)^-1
Leontif_Inverse = np.linalg.inv(np.matrix(np.identity(4863) - A))

##### Check whether (I-A)^-1 * final demand = gross output

In [250]:
final_demand = np.array(Export["TFD"])
Leontif_Inverse * np.matrix(final_demand).T

matrix([[4.52040096e+06],
        [1.84240722e+05],
        [2.79432667e+05],
        ...,
        [5.72923587e+05],
        [7.02661488e+05],
        [4.25034744e+08]])

In [251]:
calculated_output = (Leontif_Inverse * np.matrix(final_demand).T).T.tolist()[0]

In [252]:
diff = []
for i in range(len(calculated_output)):
    diff.append(calculated_output[i] - gross_output[i])

In [253]:
np.mean(diff)

2.447933629881064e-07